In [1]:
using JLD2
using Statistics
using Revise
using Plots 
using Laplacians

In [2]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [3]:
@load "uni_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic1 = copy(dic)
@load "uni_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic2 = copy(dic)
@load "uni_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic3 = copy(dic)
@load "uni_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic4 = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [Inf, Inf, Inf, 11.5191, 250.959, 164.867, Inf, Inf]
  "acM2C2_err"        => [5.3766e-9, 6.22972e-9, 5.10428e-9, 8.3241e-9, 7.35432…
  "hypre_solve"       => [114.8, 163.99, Inf, 125.44, 319.06, 115.77, 181.56, 2…
  "acM2C2_solve"      => [90.5187, 124.18, 106.85, 101.285, 314.261, 149.967, 2…
  "icc2_tot"          => [566.791, 110.233, 101.325, 197.025, 436.94, 4998.89, …
  "acM2C2_tot"        => [166.28, 274.749, 202.467, 188.58, 819.113, 298.232, 5…
  "ac_lst_build"      => [70.4133, 128.218, 106.649, 82.4338, 293.434, 119.623,…
  "petsc_hypre_build" => [Inf, 985.023, Inf, 767.596, Inf, 402.708, Inf, Inf]
  "lamg_its"          => [Inf, Inf, Inf, 2.0, 9.0, 9.0, Inf, Inf]
  "lamg_solve"        => [Inf, Inf, Inf, 0.027887, 286.42, 154.003, Inf, Inf]
  "hypre_its"         => [12.0, 15.0, Inf, 14.0, 16.0, 12.0, 11.0, 20.0]
  "cmg_solve"         => [173.181, 198.855, 267.181, 171.128, 2058.44, 231.881,…
  "ac_tot"            => [12

In [4]:
nanErr!(dic1);
nanErr!(dic2);
nanErr!(dic3);
nanErr!(dic4);

In [5]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [6]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [7]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(nfK(d["nv"][1]), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for d in dics
        printOneLine(d, solvers)
    end
end

printAllLines (generic function with 1 method)

In [8]:
dics = [dic1, dic2, dic3, dic4];

In [9]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

10&103&0.421&0.467&0.576&0.687&0.804&1.24&8.36&10.8&25.3\\
100&105&0.676&0.87&1.56&1.08&1.44&3.18&2.06&2.74&Inf\\
1000&23&1.5&1.73&2.39&1.72&2.83&3.58&2.18&2.94&12.9\\
10000&8&2.79&3.4&3.95&3.57&5.06&7.19&3.35&4.97&Inf\\


In [10]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

10&103&0.919&1.31&2.86\tnote{**}&5.86&11.9&38.8\tnote{*}&8.43&10.8&17.8\tnote{*}\\
100&105&1.38&2.29&6.23\tnote{**}&18.9&48.9&182\tnote{*}&3.35&4.9&19.2\tnote{*}\\
1000&23&3.19&5.73&15.4&97.6&370\tnote{*}&Inf&6.92&11.3\tnote{*}&28.4\tnote{*}\\
10000&8&6.7&7.8&Inf&Inf&Inf&Inf&4.3&9.52\tnote{*}&41\tnote{*}\\
